### Liquid Clustering

- Liquid clustering in Databricks is a feature that dynamically reorganizes data within a Delta Lake table to improve query performance, acting as a replacement for traditional methods like static partitioning and Z-Ordering. It intelligently organizes data inside files, optimizing it for read-time data skipping and automatic optimization of incremental data, reducing the need for manual table rewrites as data patterns change. 

**Documentation**: 
- [Use liquid clustering for tables](https://learn.microsoft.com/en-us/azure/databricks/delta/clustering)
- [How Delta Lake Liquid Clustering conceptually works ](https://dennyglee.com/2024/02/06/how-delta-lake-liquid-clustering-conceptually-works/)


In [0]:
df1 = spark.read.parquet("dbfs:/FileStore/data/shopping/invoices_101_200.parquet")
df2 = spark.read.parquet("dbfs:/FileStore/data/shopping/invoices_1_100.parquet")
df3 = spark.read.parquet("dbfs:/FileStore/data/shopping/invoices_201_99457.parquet")

In [0]:
df_union = df1.union(df2).union(df3)
df_union = df_union.select("customer_id", "category", "price", "quantity", "invoice_date")

- **Create table using hive-style partitioning and optimize it**

In [0]:
df_union.write.mode("overwrite").partitionBy("invoice_date").saveAsTable("lc_ex1")

In [0]:
%sql
OPTIMIZE lc_ex1 ZORDER BY customer_id; 

In [0]:
%sql
SELECT count(*) FROM lc_ex1; 

%md
- **Create table using liquid clustering**

In [0]:
df_union.write.mode("overwrite").clusterBy("invoice_date", "customer_id").saveAsTable("lc_ex2")

In [0]:
%sql
SELECT count(*) FROM lc_ex2; 

In [0]:
%%time
spark.sql(
    """
    SELECT category, SUM(price * quantity) AS total_sales
    FROM 
        lc_ex1
    WHERE 
        (invoice_date BETWEEN '2021-01-01' AND '2023-12-31') AND customer_id = 201
    GROUP BY 
        category
    """
)

In [0]:
%%time
spark.sql(
    """
    SELECT category, SUM(price * quantity) AS total_sales
    FROM
         lc_ex2
    WHERE 
        (invoice_date BETWEEN '2021-01-01' AND '2023-12-31') AND customer_id = 201
    GROUP BY 
        category
    """
)